In [ ]:
import pandas as pd
import mysql.connector

conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Admin@123",
                               database = "dataspark")

cursor = conn.cursor()

Customer Analysis

In [ ]:
#gender distribution
gender_distribution = '''select gender,count(DISTINCT customerkey) as count 
                         from combined_sales
                        group by gender;'''
cursor.execute(gender_distribution)
result = cursor.fetchall()
df_gender_distribution = pd.DataFrame(result, columns=['gender', 'count'])
print(df_gender_distribution)

#age_distribution
age_distribution = '''select age,count(DISTINCT customerkey) as count
from combined_sales
group by age
order by age;'''
cursor.execute(age_distribution)
result = cursor.fetchall()
df_age_distribution = pd.DataFrame(result, columns=['Age', 'count'])
print(df_age_distribution.head())

#Customer Segmentation
Customer_Segmentation = '''select segment,count(DISTINCT customerkey) as count
from combined_sales
group by segment
order by count desc;'''
cursor.execute(Customer_Segmentation)
result = cursor.fetchall()
df_Customer_Segmentation = pd.DataFrame(result, columns=['Segment', 'count'])
print(df_Customer_Segmentation.head())

#Lifetime Value
Lifetime_Value = '''SELECT customerkey, 
       customer_name, 
       ROUND(SUM(unit_price_usd * exchange * quantity), 2) AS total_revenue
FROM combined_sales
GROUP BY customerkey, customer_name
ORDER BY total_revenue DESC;'''
cursor.execute(Lifetime_Value)
result = cursor.fetchall()
df_Lifetime_Value = pd.DataFrame(result, columns=['customerkey', 'customer_name','total_revenue'])
print(df_Lifetime_Value.head())

#Top 10Lifetime Value
Top_10_Lifetime_Value = '''SELECT customerkey, 
       customer_name, 
       ROUND(SUM(unit_price_usd * exchange * quantity), 2) AS total_revenue
FROM combined_sales
GROUP BY customerkey, customer_name
ORDER BY total_revenue DESC
limit 100;'''
cursor.execute(Top_10_Lifetime_Value)
result = cursor.fetchall()
df_Top_10_Lifetime_Value = pd.DataFrame(result, columns=['customerkey', 'customer_name','total_revenue'])
df_Top_10_Lifetime_Value.head()

Sales and Revenue Analysis

In [ ]:
#Total Revenue
Total_Revenue = '''SELECT ROUND(SUM(unit_price_usd * exchange * quantity), 2) AS total_revenue
FROM combined_sales;'''
cursor.execute(Total_Revenue)
result = cursor.fetchall()
df_Total_Revenue = pd.DataFrame(result, columns=['total_revenue'])
print(df_Total_Revenue.head())

In [ ]:
#Monthly Sales
Monthly_Sales = '''SELECT DATE_FORMAT(order_date, '%Y-%m') AS month, ROUND(SUM(unit_price_usd * exchange * quantity), 2) AS total_revenue
FROM combined_sales 
GROUP BY DATE_FORMAT(order_date, '%Y-%m')
ORDER BY month;'''
cursor.execute(Monthly_Sales)
result = cursor.fetchall()
df_Monthly_Sales = pd.DataFrame(result, columns=['month','total_revenue'])
print(df_Monthly_Sales.head())

In [ ]:
#Top 10 Products
Top_Products ='''SELECT product_name, SUM(quantity) AS total_quantity, ROUND(SUM(unit_price_usd * exchange * quantity), 2) AS total_revenue
FROM combined_sales 
GROUP BY product_name
ORDER BY total_revenue DESC
LIMIT 10;'''
cursor.execute(Top_Products)
result = cursor.fetchall()
df_Top_Products = pd.DataFrame(result, columns=['product_name','total_quantity','total_revenue'])
print(df_Top_Products.head())

Pricing and Profitability Analysis

In [ ]:
#Unit Cost vs. Unit Price
cost_vs_price = '''SELECT product_name, unit_cost_usd, unit_price_usd, (unit_price_usd - unit_cost_usd) AS profit_margin
FROM combined_sales;'''
cursor.execute(cost_vs_price)
result = cursor.fetchall()
df_cost_vs_price = pd.DataFrame(result, columns=['product_name','unit_cost_usd','unit_price_usd','profit_margin'])
print(df_cost_vs_price.head())

Order Analysis

In [35]:
#Number of Orders
total_orders = '''SELECT COUNT(DISTINCT order_number) AS total_orders
FROM combined_sales;'''
cursor.execute(total_orders)
result = cursor.fetchall()
df_total_orders = pd.DataFrame(result, columns=['total_orders'])
print(df_total_orders.head())

   total_orders
0         26326


In [37]:
#Average Order Value
Average_Order_Value=''' 
SELECT round(avg(total_revenue),2) AS average_order_value
FROM (
    SELECT order_number, ROUND(SUM(unit_price_usd * exchange * quantity), 2) AS total_revenue
    FROM combined_sales
    GROUP BY order_number
) sub;'''
cursor.execute(Average_Order_Value)
result = cursor.fetchall()
df_average_order_value = pd.DataFrame(result, columns=['average_order_value'])
print(df_average_order_value.head())

  average_order_value
0             2102.46


Customer Behavior and Loyalty

In [ ]:
#Purchase Frequency
Purchase_Frequency = '''SELECT customerkey, customer_name, COUNT(DISTINCT order_number) AS purchase_frequency
FROM combined_sales
GROUP BY customerkey, customer_name
ORDER BY purchase_frequency DESC;'''
cursor.execute(Purchase_Frequency)
result = cursor.fetchall()
df_Purchase_Frequency = pd.DataFrame(result, columns=['customerkey','customer_name','purchase_frequency'])
print(df_Purchase_Frequency.head())

In [ ]:
#Churn Analysis
Churn_Analysis = '''SELECT customerkey, customer_name, MAX(order_date) AS last_purchase_date
FROM combined_sales 
GROUP BY customerkey, customer_name
HAVING MAX(order_date) < DATE_SUB(CURDATE(), INTERVAL 6 MONTH);'''
cursor.execute(Churn_Analysis)
result = cursor.fetchall()
df_Churn_Analysis = pd.DataFrame(result, columns=['customerkey','customer_name','last_purchase_date'])
print(df_Churn_Analysis.head())

Sales by Location

In [ ]:
#Sales by Country
Sales_by_Country = '''SELECT country, ROUND(SUM(unit_price_usd * exchange * quantity), 2) AS total_revenue
FROM combined_sales 
GROUP BY country
ORDER BY total_revenue DESC;
'''
cursor.execute(Sales_by_Country)
result = cursor.fetchall()
df_Sales_by_Country = pd.DataFrame(result, columns=['country','total_revenue'])
print(df_Sales_by_Country.head())

In [41]:
#sales by store sq mtr
sales_by_store_sq_mtr = '''SELECT store_state, square_meters, ROUND(SUM(unit_price_usd * exchange * quantity), 2) AS total_revenue
FROM combined_sales 
GROUP BY store_state,square_meters
ORDER BY total_revenue DESC;'''
cursor.execute(sales_by_store_sq_mtr)
result = cursor.fetchall()
df_sales_by_store_sq_mtr = pd.DataFrame(result, columns=['store_state','square_meters','total_revenue'])
print(df_sales_by_store_sq_mtr.head())

                 store_state square_meters total_revenue
0                     Online          0.00   11431369.95
1      Northwest Territories       1500.00    1756645.28
2  Newfoundland and Labrador       2105.00    1617620.45
3                     Nevada       2000.00    1417885.41
4                     Kansas       2000.00    1394738.06
